In [6]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [8]:
import pandas as pd
import sys, re
import numpy as np

In [11]:
train_pd = pd.read_csv('./drive/MyDrive/Colab_Notebooks/Technopark_ML/DZ_3/train.csv')

In [7]:
# (Токенизация/лемматизация на результат особо не повлияли)

def clean_str(string):
    string_cl = re.sub(r'[0-9-—–.,()@|\/*«»:;_!?\'\"№&#•⋆<>{}\]\[]', "", string)     
    return string_cl.strip().lower() 

In [33]:
def clean_data(data, labels_flag=False):
  data_norm = []
  labels = []

  for index, row in data.iterrows():
    
    data_norm.append(clean_str(row['title']))

    if labels_flag==True:
      if row['target']==True:
        labels.append(1)
      else:
        labels.append(0)
  return data_norm, labels    

In [13]:
# ----------------------------------- Обучение -----------------------------------

In [5]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):

    data = np.array(data)
    data_size = len(data)
    print(data_size)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [4]:
# Идея свистнута из статьи: http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/

import tensorflow as tf

class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")     
      

In [15]:
# Параметры данных
dev_sample_percentage = 0.1

# Параметры модели
embedding_dim = 128
filter_sizes = '3,4,5'
num_filters=128
dropout_keep_prob=0.5
l2_reg_lambda = 0.0

# Параметры обучения
batch_size = 250 
num_epochs = 100
evaluate_every = 100
checkpoint_every = 250
num_checkpoints = 5

allow_soft_placement = True
log_device_placement = False


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.13.2

# (Пытаемся вытащить tensorflow.contrib)

In [ ]:
from tensorflow.contrib import learn

def preprocess(data_norm, labels):

    print("Loading data...")
    x_text = data_norm
    y = labels

    # Собираем словарь
    max_document_length = max([len(x.split()) for x in x_text])
    print("len = ", max_document_length)

    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    np.random.seed(42)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = np.array(x)[shuffle_indices]
    y_shuffled = np.array(y)[shuffle_indices]


    dev_sample_index = -1 * int(dev_sample_percentage * float(len(y)))
    print("dev_sample_index = ", dev_sample_index)
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev

In [ ]:
data_norm, labels = clean_data(train_pd, labels_flag=True)
x_train, y_train, vocab_processor, x_dev, y_dev = preprocess(data_norm, labels)

In [2]:
import time
import datetime
import os


def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    # Training
    # ==================================================
    path = './drive/MyDrive/Colab_Notebooks/Technopark_ML/DZ_3/model'

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=allow_soft_placement,
          log_device_placement=log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=embedding_dim,
                filter_sizes=list(map(int, filter_sizes.split(","))),
                num_filters=num_filters,
                l2_reg_lambda=l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(path, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train)), batch_size, num_epochs)
            # Training loop. For each batch...
            
            for batch in batches: 
                # print(batch)
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

In [ ]:
# Обучение

# train(x_train, y_train, vocab_processor, x_dev, y_dev)

In [4]:
# -------------------------- тестирование на train-данных-------------------------------------

In [10]:
# Берем сохраненную обученную модель

path = './drive/MyDrive/Colab_Notebooks/Technopark_ML/DZ_3/model'
checkpoint_dir = './drive/MyDrive/Colab_Notebooks/Technopark_ML/DZ_3/model/runs/1636913449/checkpoints'

vocab_path = os.path.join(checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)

checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)

In [13]:
def predict(data):

  x_test = np.array(list(vocab_processor.transform(data)))
  print(len(x_test))

  graph = tf.Graph()
  with graph.as_default():
      session_conf = tf.ConfigProto(
        allow_soft_placement=allow_soft_placement,
        log_device_placement=log_device_placement)
      sess = tf.Session(config=session_conf)
      with sess.as_default():
          # Грузим сохраненные переменные
          saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
          saver.restore(sess, checkpoint_file)

          input_x = graph.get_operation_by_name("input_x").outputs[0]
          dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

          predictions = graph.get_operation_by_name("output/predictions").outputs[0]

          batches = batch_iter(list(x_test), batch_size, 1, shuffle=False)

          all_predictions = []

          for x_test_batch in batches:
              batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
              all_predictions = np.concatenate([all_predictions, batch_predictions])
    
  return all_predictions          

In [26]:
data_norm, labels = clean_data(train_pd, labels_flag=True)
all_predictions = predict(data_norm)

In [17]:
len(all_predictions)

135309

In [28]:
# Оценка результатов
from sklearn.metrics import f1_score


if labels is not None:
    correct_predictions = float(sum(all_predictions == labels))
    print("Total number of test examples: {}".format(len(labels)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(labels))))

    print("F1 = ", f1_score(labels, all_predictions))


Total number of test examples: 135309
Accuracy: 0.996637
F1 =  0.9863449476306233


In [ ]:
# ------------------------Оценка данных из test.csv-----------------------

In [29]:
test_pd = pd.read_csv('./drive/MyDrive/Colab_Notebooks/Technopark_ML/DZ_3/test.csv')

In [32]:
len(test_pd)

165378

In [36]:
x_raw_test, _ = clean_data(test_pd)

In [37]:
len(x_raw_test)

165378

In [38]:
all_pred_np = np.asarray(all_predictions)
all_pred_bool = np.equal(all_pred_np, True)
ids = test_pd['id']

In [39]:
ids = ids.to_frame()
pred_ser = pd.Series(all_pred_bool)
ids['label'] = pred_ser

In [ ]:
ids.to_csv(index=False, path_or_buf='./drive/MyDrive/Colab_Notebooks/Technopark_ML/DZ_3/res.csv')